In [ ]:
# # Internship Task 1: Big Data Analysis using PySpark
# This notebook analyzes a large fraud detection dataset using PySpark to demonstrate scalability.


In [1]:
# 🔧 Step 1: Install Java (Spark needs Java runtime)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# 🔧 Step 2: Download Spark 3.5.1 (you can change version if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

# 🔧 Step 3: Extract Spark package
!tar -xzf spark-3.5.1-bin-hadoop3.tgz

# 🔧 Step 4: Install findspark to connect Python with Spark
!pip install -q findspark


In [2]:
import os
import findspark

# Set environment paths for Java and Spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

# Initialize findspark
findspark.init()


In [3]:
from google.colab import files
uploaded = files.upload()


Saving fraud_detection_bank_dataset.csv to fraud_detection_bank_dataset.csv


In [6]:
from pyspark.sql import SparkSession

# Start or get the SparkSession
spark = SparkSession.builder \
    .appName("My Fraud Detection Analysis") \
    .getOrCreate()


In [7]:
df = spark.read.csv("fraud_detection_bank_dataset.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)


root
 |-- _c0: integer (nullable = true)
 |-- col_0: integer (nullable = true)
 |-- col_1: integer (nullable = true)
 |-- col_2: integer (nullable = true)
 |-- col_3: integer (nullable = true)
 |-- col_4: integer (nullable = true)
 |-- col_5: integer (nullable = true)
 |-- col_6: integer (nullable = true)
 |-- col_7: integer (nullable = true)
 |-- col_8: integer (nullable = true)
 |-- col_9: integer (nullable = true)
 |-- col_10: integer (nullable = true)
 |-- col_11: integer (nullable = true)
 |-- col_12: integer (nullable = true)
 |-- col_13: integer (nullable = true)
 |-- col_14: integer (nullable = true)
 |-- col_15: integer (nullable = true)
 |-- col_16: integer (nullable = true)
 |-- col_17: integer (nullable = true)
 |-- col_18: integer (nullable = true)
 |-- col_19: integer (nullable = true)
 |-- col_20: integer (nullable = true)
 |-- col_21: integer (nullable = true)
 |-- col_22: integer (nullable = true)
 |-- col_23: integer (nullable = true)
 |-- col_24: integer (nullable = 

In [8]:
print("Row count:", df.count())
print("Column count:", len(df.columns))


Row count: 20468
Column count: 114


In [9]:
df.describe().show()


+-------+-----------------+------------------+------------------+-------------------+-----------------+-------------------+------------------+-----------------+------------------+-----+-----+------+------+------+-------------------+-------------------+-------------------+-------------------+------------------+------+------+------+------+------------------+-----------------+------------------+------------------+-----------------+-------------------+--------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+------+------------------+-----------------+-----------------+--------------------+-------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+-----------------+------------------+------------------+------------------+------+------+------+------------------+------------------+------------------+------------------+--------------------+---

In [10]:
from pyspark.sql.functions import sum

# Get columns where all values = 0
zero_cols = []
for col_name in df.columns:
    if df.select(sum(col_name)).first()[0] == 0:
        zero_cols.append(col_name)

# Drop them
df_clean = df.drop(*zero_cols)
print("Dropped columns:", zero_cols)


Dropped columns: ['col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_18', 'col_19', 'col_20', 'col_21', 'col_35', 'col_51', 'col_52', 'col_53', 'col_70', 'col_71']


In [11]:
df_clean.columns


['_c0',
 'col_0',
 'col_1',
 'col_2',
 'col_3',
 'col_4',
 'col_5',
 'col_6',
 'col_7',
 'col_13',
 'col_14',
 'col_15',
 'col_16',
 'col_17',
 'col_22',
 'col_23',
 'col_24',
 'col_25',
 'col_26',
 'col_27',
 'col_28',
 'col_29',
 'col_30',
 'col_31',
 'col_32',
 'col_33',
 'col_34',
 'col_36',
 'col_37',
 'col_38',
 'col_39',
 'col_40',
 'col_41',
 'col_42',
 'col_43',
 'col_44',
 'col_45',
 'col_46',
 'col_47',
 'col_48',
 'col_49',
 'col_50',
 'col_54',
 'col_55',
 'col_56',
 'col_57',
 'col_58',
 'col_59',
 'col_60',
 'col_61',
 'col_62',
 'col_63',
 'col_64',
 'col_65',
 'col_66',
 'col_67',
 'col_68',
 'col_69',
 'col_72',
 'col_73',
 'col_74',
 'col_75',
 'col_76',
 'col_77',
 'col_78',
 'col_79',
 'col_80',
 'col_81',
 'col_82',
 'col_83',
 'col_84',
 'col_85',
 'col_86',
 'col_87',
 'col_88',
 'col_89',
 'col_90',
 'col_91',
 'col_92',
 'col_93',
 'col_94',
 'col_95',
 'col_96',
 'col_97',
 'col_98',
 'col_99',
 'col_100',
 'col_101',
 'col_102',
 'col_103',
 'col_104',
 'col

In [12]:
new_names = [
    "amount", "oldbalanceOrg", "newbalanceOrig", "type",  # first few known fields
    # Add generic names for the rest
] + [f"feature_{i}" for i in range(len(df_clean.columns) - 5)] + ["isFraud"]  # assuming last column is the target

df_clean = df_clean.toDF(*new_names)


In [13]:
df_clean.show(5)


+------+-------------+--------------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------

In [14]:
print("Rows:", df_clean.count())
print("Columns:", len(df_clean.columns))


Rows: 20468
Columns: 99


In [15]:
df_clean.printSchema()

root
 |-- amount: integer (nullable = true)
 |-- oldbalanceOrg: integer (nullable = true)
 |-- newbalanceOrig: integer (nullable = true)
 |-- type: integer (nullable = true)
 |-- feature_0: integer (nullable = true)
 |-- feature_1: integer (nullable = true)
 |-- feature_2: integer (nullable = true)
 |-- feature_3: integer (nullable = true)
 |-- feature_4: integer (nullable = true)
 |-- feature_5: integer (nullable = true)
 |-- feature_6: integer (nullable = true)
 |-- feature_7: integer (nullable = true)
 |-- feature_8: integer (nullable = true)
 |-- feature_9: integer (nullable = true)
 |-- feature_10: integer (nullable = true)
 |-- feature_11: integer (nullable = true)
 |-- feature_12: integer (nullable = true)
 |-- feature_13: integer (nullable = true)
 |-- feature_14: integer (nullable = true)
 |-- feature_15: integer (nullable = true)
 |-- feature_16: integer (nullable = true)
 |-- feature_17: integer (nullable = true)
 |-- feature_18: integer (nullable = true)
 |-- feature_19: in

In [16]:
df_clean.groupBy("isFraud").count().show()  #count farud transaction

+-------+-----+
|isFraud|count|
+-------+-----+
|      1| 5438|
|      0|15030|
+-------+-----+



In [17]:
df_clean.filter("isFraud == 1").groupBy().avg("amount").show() # avg transation amount for fraud

+-----------+
|avg(amount)|
+-----------+
|     2718.5|
+-----------+



In [18]:
df_clean.groupBy("type", "isFraud").count().orderBy("type").show() # fraud count by transaction

+----+-------+-----+
|type|isFraud|count|
+----+-------+-----+
|   0|      0|12826|
|   0|      1| 5276|
|   1|      0| 1119|
|   1|      1|  102|
|   2|      1|   25|
|   2|      0|  611|
|   3|      1|    8|
|   3|      0|  170|
|   4|      0|   66|
|   4|      1|   15|
|   5|      1|    4|
|   5|      0|   40|
|   6|      0|   40|
|   6|      1|    2|
|   7|      1|    1|
|   7|      0|   26|
|   8|      0|   15|
|   9|      0|   17|
|  10|      0|    9|
|  11|      0|    8|
+----+-------+-----+
only showing top 20 rows



In [ ]:
# Insights:
# - Transaction Type 0 had the highest volume and most frauds.
# - Fraud was present across nearly all types.
# - Type 1 had a significant fraud rate.
# - 15 columns were dropped due to containing only 0s.

# This shows the dataset had redundant information, and PySpark allowed fast filtering and analysis even for large structured data.
